In [2]:
import os
import sys
import pandas as pd
import pipe

In [3]:
os.chdir("D:/Drive/FHNW/zaRepos/fhnw_ds_fs2021_medical_challenge")
rel_path = "./data/uveitis_data.xlsx"

In [4]:
#function that reads data

df1 = pipe.read_data(rel_path=rel_path)

def read_data(rel_path, **kwargs):
    """
    Generalized function that reads a xlsx, csv, json or html data structure into a pandas dataframe

    :argument rel_path: relative path to data to be read
    :argument kwargs:  parameters for pd.read_"extension" functions
    """
    filename, file_extension = os.path.splitext(rel_path)
    print(os.path.basename(rel_path) + " will be read")
    if file_extension == ".xlsx": #when reading excel a usefull kwargs will be na_values = "dict of values to consider na"
        df = pd.read_excel(rel_path, **kwargs)
        print("reading filetype "+file_extension)
    elif file_extension == ".csv":
        df = pd.read_csv(rel_path, **kwargs)
        print("reading filetype "+file_extension)
    elif file_extension == ".json":
        df = pd.read_json(rel_path, **kwargs)
        print("reading filetype "+file_extension)
    elif file_extension == ".html":
        df = pd.read_html(rel_path, **kwargs)
        print("reading filetype "+file_extension)
    else:
        print("Filetype not supported by function")
        df = 0

    if df.size != 0:
        print("\nSuccessfully created table with ", df.size, "values and loaded as df")
        print("The table is",df.shape[1], "wide and",df.shape[0],"long \n")

    return df

df1 = read_data(rel_path)

AttributeError: module 'pipe' has no attribute 'read_data'

In [5]:
#function to return list of columns to convert to which data type (with choice) according to given list
col_name_data_type = "data_type"
col_index= "new col name"
list_path = "./data/col_names&data_type-Copy1.xlsx"
data_type = "numerical"

def list_of_totype(list_path, col_index, col_name_data_type, data_type):
    """
    :param list_path: path of excel document to read data type from
    :param col_index: index column to use (column name of data type)
    :param col_name_data_type: name of column where data type is stored
    :param data_type: type of data desired. Should be numerical, categorical, char or both (appelation for edge cases)
    :return: returns a list of column names with the desired data type
    """
    col = pd.read_excel(list_path, index_col = col_index)[col_name_data_type] #get content of column with name data_type
    col_to_type = col.where(col == data_type).dropna().index.tolist()
    return col_to_type

#item_filter = list_of_totype(list_path, col_index, col_name_data_type, data_type)
#print(item_filter)

In [28]:
#function that should return all values that couldn't be correctly coerced by next function (to be used in another function)
desired_dtype = ["Int64", "float64"]

def problem_columns(matches, desired_dtype):
    """
    Return all columns and their content that couldn't be correctly coerced
    :param matches: feeded from another function
    :param desired_dtype: a list or string that contains the desired data_types that were transformed correctly
    :return: returns a dataframe with all the problematic columns
    """
    for key, value in matches.iteritems():
        if value.dtype in desired_dtype:
            print("\n", key, "is desired", value.dtype, "and will be popped from problematic list \n")
            matches.pop(key)
        else:
            print(key, "has unwanted dtype, keeping for transformation")
    return matches

In [29]:
#function to coerce columns to desired datatype
def coerce_then_problems(dframe, list_path, col_index, col_name_data_type, data_type, desired_dtype):
    """
    :param dframe: dataframe to be checked and wrecked
    :param list_path: path of excel document to read data type from
    :param col_index: index column to use (column name of data type)
    :param col_name_data_type: name of column where data type is stored
    :param data_type: type of data desired. Should be numerical, categorical, char or both (appelation for edge cases)
    :param desired_dtype: a list or string that contains the desired data_types that were transformed correctly
    :return: a dataframe with values that couldn't be coerced to the desired data_type automagically via convert_dtypes()
    """

    item_filter = list_of_totype(list_path, col_index, col_name_data_type, data_type)
    matches = dframe.filter(items = item_filter).convert_dtypes()
    print(matches)
    matches = problem_columns(matches, desired_dtype)
    return matches

In [30]:
problem_df = coerce_then_problems(df1, list_path, col_index, col_name_data_type, data_type, desired_dtype)
problem_columns = list(problem_df)
print(problem_columns)
print(type(problem_df))

#function that iterates trough columns given by previous function to coerce values that aren't numerical in nature to something else (0, , NaN, NA, 99)


      SUBJECT ID Calcium (Blood) Lactate Dehydrogenase (LDH) (Blood)  \
0              1            2.27                                 184   
1              2            2.32                                 209   
2              3            2.44                                 194   
3              4            2.45                                 156   
4              5            2.28                                 175   
...          ...             ...                                 ...   
1070        1071            2.26                                 194   
1071        1072            2.36                                 239   
1072        1073            2.26                                 196   
1073        1074            2.41                                 173   
1074        1075            2.39                                 214   

     C-Reactive Protein, Normal and High Sensitivity (Blood) WBC (Blood)  \
0                                                  1.83    

C:\Users\Lupo_Remastered\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  # This is added back by InteractiveShellApp.init_path()


In [31]:
def extract_num(df, column, errors='coerce', verbose=False):
    """
    Extracts numerical values out of a column and returns the DataFrame with said column in numerical dtype

    Arguments
    ---------
    df: pandas.core.frame.DataFrame, Pandas DataFrame
    column: str, Name of column whos numerical values should be extracted
    errors: str, Determines how the function should handle errors (i.e nan's etc.), handled by pandas.to_numeric
            - If 'raise', then invalid parsing will raise an exception.
            - If 'coerce', then invalid parsing will be set as NaN.
            - If 'ignore', then invalid parsing will return the input.

    Returns
    -------
    df: pandas.core.frame.DataFrame
    """
    if verbose:
        print("Unique values before transformation:")
        print(df[column].unique())

    # filter out decimal numbers of column
    df[column] = df[column].astype('string')
    df[column] = df[column].str.extract(r"(\d{1,}[.|,]\d{1,}|\d*)", expand=False)
    df[column] = pd.to_numeric(df[column], errors=errors)

    if verbose:
        print("Unique values after transformation:")
        print(df[column].unique())

    return df

extract_num(problem_df, 'Calcium (Blood)', errors='coerce', verbose=False)

ValueError: StringArray requires a sequence of strings or pandas.NA